## Churn Analysis

In [1]:
import pandas as pd
import datetime as dt

In [2]:
# Load data
data = pd.read_csv('../data/online_retail_cleaned.csv')
data.shape

(406829, 9)

In [3]:
# Convert 'InvoiceDate' to datetime
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

# Extract returns
data_returns = data[(data['Quantity'] <= 0) | (data['UnitPrice'] <= 0)].copy()
print("Number of returns:", data_returns.shape)

# Remove returns from the dataset
data_purchases = data[(data['Quantity'] > 0) & (data['UnitPrice'] > 0)].copy()
print("Number of purchases:", data_purchases.shape)

Number of returns: (8945, 9)
Number of purchases: (397884, 9)


## Compute churn and features
Churn definition: Customer who hasn't made a purchase in 60 days

In [4]:
# Snapshot date 
snapshot_date = data_purchases['InvoiceDate'].max() + dt.timedelta(days=1)
print("Snapshot date:", snapshot_date)

Snapshot date: 2011-12-10 12:50:00


In [5]:
# Last purchase per customer and churn calculation
last_purchase = (
    data_purchases
    .groupby('CustomerID')['InvoiceDate']
    .max()
    .reset_index()
    .rename(columns={'InvoiceDate': 'LastPurchaseDate'})
)
last_purchase['Recency'] = (snapshot_date - last_purchase['LastPurchaseDate']).dt.days
last_purchase['Churn'] = (last_purchase['Recency'] > 60).astype(int)

print("Churn rate:", last_purchase['Churn'].mean())
last_purchase.head()

Churn rate: 0.4476717381281697


,CustomerID,LastPurchaseDate,Recency,Churn
0,12346.0,2011-01-18 10:01:00,326,1
1,12347.0,2011-12-07 15:52:00,2,0
2,12348.0,2011-09-25 13:13:00,75,1
3,12349.0,2011-11-21 09:51:00,19,0
4,12350.0,2011-02-02 16:01:00,310,1


In [6]:
# Frequency (number of purchases)
frequency = data_purchases.groupby('CustomerID')['InvoiceNo'].nunique().reset_index()
frequency.columns = ['CustomerID', 'Frequency']
frequency.head()

,CustomerID,Frequency
0,12346.0,1
1,12347.0,7
2,12348.0,4
3,12349.0,1
4,12350.0,1


In [7]:
# Monetary - total revenue and average spent per customer
data['Revenue'] = data['Quantity'] * data['UnitPrice']
# 1. Calculate total revenue per invoice
invoice_totals = data.groupby(['CustomerID', 'InvoiceNo'])['Revenue'].sum().reset_index()
# 2. Calculate total and average revenue per customer
monetary = invoice_totals.groupby('CustomerID')['Revenue'].agg(['sum', 'mean']).reset_index()
monetary.columns = ['CustomerID', 'TotalRevenue', 'AvgOrderValue']
monetary.head()

,CustomerID,TotalRevenue,AvgOrderValue
0,12346.0,0.00,0.000000
1,12347.0,4310.00,615.714286
2,12348.0,1797.24,449.310000
3,12349.0,1757.55,1757.550000
4,12350.0,334.40,334.400000


In [8]:
# Number of returns per customer
num_returns = data_returns.groupby('CustomerID')['Quantity'].count().reset_index()
num_returns.columns = ['CustomerID', 'NumReturns']
num_returns.head()

,CustomerID,NumReturns
0,12346.0,1
1,12352.0,10
2,12359.0,6
3,12362.0,8
4,12365.0,1


In [9]:
# Total return value per customer
return_revenue = data_returns.groupby('CustomerID')['Revenue'].sum().reset_index()
return_revenue.columns = ['CustomerID', 'ReturnRevenue']
return_revenue.head()

,CustomerID,ReturnRevenue
0,12346.0,-77183.60
1,12352.0,-960.63
2,12359.0,-127.05
3,12362.0,-71.65
4,12365.0,-320.69


In [10]:
# Product diversity (number of unique products purchased)
product_diversity = data_purchases.groupby('CustomerID')['StockCode'].nunique().reset_index()
product_diversity.columns = ['CustomerID', 'ProductDiversity']  
product_diversity.head()

,CustomerID,ProductDiversity
0,12346.0,1
1,12347.0,103
2,12348.0,22
3,12349.0,73
4,12350.0,17


In [11]:
# Geography (Country)
country = data.groupby('CustomerID')['Country'].first().reset_index()
country.columns = ['CustomerID', 'Country']
country.head()

,CustomerID,Country
0,12346.0,United Kingdom
1,12347.0,Iceland
2,12348.0,Finland
3,12349.0,Italy
4,12350.0,Norway


In [12]:
# Merge all together
customer_features = last_purchase.merge(frequency, on='CustomerID', how='left')
customer_features = customer_features.merge(monetary, on='CustomerID', how='left')
customer_features = customer_features.merge(num_returns, on='CustomerID', how='left')
customer_features = customer_features.merge(return_revenue, on='CustomerID', how='left')
customer_features = customer_features.merge(product_diversity, on='CustomerID', how='left')
customer_features = customer_features.merge(country, on='CustomerID', how='left')

# Fill NaNs for customers with no returns
customer_features['NumReturns'] = customer_features['NumReturns'].fillna(0)
customer_features['ReturnRevenue'] = customer_features['ReturnRevenue'].fillna(0)


In [13]:
customer_features

,CustomerID,LastPurchaseDate,Recency,Churn,Frequency,TotalRevenue,AvgOrderValue,NumReturns,ReturnRevenue,ProductDiversity,Country
0,12346.0,2011-01-18 10:01:00,326,1,1,0.00,0.000000,1.0,-77183.60,1,United Kingdom
1,12347.0,2011-12-07 15:52:00,2,0,7,4310.00,615.714286,0.0,0.00,103,Iceland
2,12348.0,2011-09-25 13:13:00,75,1,4,1797.24,449.310000,0.0,0.00,22,Finland
3,12349.0,2011-11-21 09:51:00,19,0,1,1757.55,1757.550000,0.0,0.00,73,Italy
4,12350.0,2011-02-02 16:01:00,310,1,1,334.40,334.400000,0.0,0.00,17,Norway
...,...,...,...,...,...,...,...,...,...,...,...
4333,18280.0,2011-03-07 09:52:00,278,1,1,180.60,180.600000,0.0,0.00,10,United Kingdom
4334,18281.0,2011-06-12 10:53:00,181,1,1,80.82,80.820000,0.0,0.00,7,United Kingdom
4335,18282.0,2011-12-02 11:43:00,8,0,2,176.60,58.866667,1.0,-1.45,12,United Kingdom
4336,18283.0,2011-12-06 12:02:00,4,0,16,2094.88,130.930000,0.0,0.00,263,United Kingdom


## Build models

## Output

### Feature importance

### ROC curve and AUC score

### Confusion matrix

### Top churn drivers with business interpretation